## Example for calculating the correlation matrix
### Calculating Color Temperature and Illuminance using the TAOS TCS3414CS Digital Color Sensor

In [194]:
import numpy as np

### Example from the TAOS Designer's Notebook [1]
***
__needed__:<br>
- S measured AD-converter values [RGB]<br>
Sensor Antwort eines RGB Sensors<br>
([<br>
  [measurement_1 R,G,B],<br>
  [measurement_2 R,G,B],<br>
  [measurement_3 R,G,B],<br>
   ...<br>
  [measurement_n R,G,B]<br>
])<br>
<p><p>__and__</p></p>
- T CIE 1931 XYZ measurement values from a spectrometer<br>
([<br>
  [measurement_1 X,Y,Z],<br>
  [measurement_2 X,Y,Z],<br>
  [measurement_3 X,Y,Z],<br>
   ...<br>
  [measurement_n X,Y,Z]<br>
 ])<br>

<p>transposing the input values is required! </p>

__[[1] AMS Calculating Color Temperature](http://ams.com/chi/content/view/download/145158)__






In [195]:
T = np.matrix([[ 26.9,  22.9,   4.9],
               [274.7, 262.2,  96.5],
               [321.8, 360.5, 335.5]])

S = np.matrix([[ 67.5,  43.0,  31.5],
              [ 226.5, 256.5,  94.5],
              [ 226.5, 415.0, 302.0]])
T = T.T
S = S.T

In [196]:
C = T*S.I #.T is for Transpose and .I is for Inverse matrix
print(C)

[[-0.14180679  1.54826167 -0.95566012]
 [-0.32477767  1.57878168 -0.73222601]
 [-0.68088285  0.76973481  0.56384112]]


In [197]:
#Test
S.T[0]*C.T

matrix([[ 26.9,  22.9,   4.9]])

***
# Real measurement data


In [198]:
spectrometer = np.matrix([
                   [5.85914E-05 , 5.74988E-05 , 3.15928E-05],
                   [3.14701E-05 , 3.77958E-05 , 0.000212881],
                   [1.0139E-05  , 3.71417E-05 , 5.26983E-06],
                   [8.22792E-05 , 6.02889E-05 , 1.55976E-06],
                   [6.11897E-05 , 2.64752E-05 , 1.45131E-07],
                   [5.93183E-05 , 5.05638E-05 , 1.21186E-05],
                   [9.12914E-05 , 8.48045E-05 , 3.67635E-05],
                   [5.88182E-05 , 5.90412E-05 , 3.90779E-05],
                   [7.57179E-05 , 7.77328E-05 , 6.23846E-05],
                   [5.9925E-05  , 3.10716E-05 , 1.41801E-06]
                 ])

RGB_Sensor = np.matrix([   
                    [101,133, 48],
                    [  0,130,284],
                    [ 11, 88, 30],
                    [171,135, 20],
                    [186, 79, 17],
                    [130,125, 33],
                    [183,207, 69],
                    [109,145, 62],
                    [131,189, 90],
                    [166, 88, 16]
                ])

spectrometer = spectrometer.T # is the same as spectrometer.transpose()
RGB_Sensor = RGB_Sensor.T

In [199]:
#after TAOS
C_M = spectrometer*RGB_Sensor.I

print(C_M)

[[  2.77611029e-07   1.90481471e-07   2.26163250e-08]
 [ -5.01731963e-08   4.89135940e-07  -9.18154796e-08]
 [ -2.73882011e-08  -5.88307647e-08   7.81347358e-07]]


In [200]:
#Test
RGB_Sensor.T[1]*C_M.T

matrix([[  3.11856275e-05,   3.75120760e-05,   2.14254650e-04]])

# Calculating Corelated Color Temperature

In [201]:
#function definition
def calc_xy(XYZ = np.matrix([9.12914E-05 , 8.48045E-05 , 3.67635E-05])):
    X = XYZ[0,0]
    Y = XYZ[0,1]
    Z = XYZ[0,2]
    
    x = X/(X+Y+Z)
    y = Y/(X+Y+Z)
    
    xy = [x,y]
    return xy
    pass

def calc_CCT(CIE_x = 0.333, CIE_y = 0.333):
    n = (CIE_x - 0.3320)/(0.1858 - CIE_y)
    CCT = 449*n**3 + 3525*n**2 + 6823.3*n + 5520.33
    return CCT

In [202]:
xy = calc_xy(RGB_Sensor.T[6]*C_M.T)
CCT = calc_CCT(xy[0],xy[1])
print('CCT = %.1f K'%CCT)

CCT = 3123.2 K


In [193]:
xy = calc_xy(S.T[2]*C.T)
CCT = calc_CCT(xy[0],xy[1])
print('CCT = %.1f K'%CCT)

CCT = 8295.5 K
